# Real-time Video

In [ ]:
import cv2

inputVideo = cv2.VideoCapture(0)
#print(type(inputVideo))
#check if connection with camera is successfully
if inputVideo.isOpened():
    ret, frame = inputVideo.read()  #capture a frame from live video

print(inputVideo.isOpened())

cameraMatrix = None
distCoeffs = None
markerLength = 0.05

# You can read camera parameters from tutorial_camera_params.yml
fs = cv2.FileStorage("tutorial_camera_params.yml", cv2.FILE_STORAGE_READ)
cameraMatrix = fs.getNode("cameraMatrix").mat()
distCoeffs = fs.getNode("distCoeffs").mat()
fs.release()

# Set coordinate system
objPoints = np.array([[-markerLength/2.0, markerLength/2.0, 0], [markerLength/2.0, markerLength/2.0, 0], [markerLength/2.0, -markerLength/2.0, 0], [-markerLength/2.0, -markerLength/2.0, 0]], dtype=np.float32)

detectorParams = cv2.aruco.DetectorParameters()
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

while True:
    ret, frame = inputVideo.read()
    if not ret:
        break
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(image, dictionary, parameters=detectorParams)
    if ids is not None and len(ids) > 0:
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, markerLength, cameraMatrix, distCoeffs)
        for rvec, tvec in zip(rvecs, tvecs):
            cv2.drawFrameAxes(frame, cameraMatrix, distCoeffs, rvec, tvec, 0.1)
    cv2.imshow("out", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

inputVideo.release()
cv2.destroyAllWindows()

## Video Input

### Intersection mean

In [ ]:
import cv2
import numpy as np
import pandas as pd

inputVideo = cv2.VideoCapture("video\whiteBorders_Trim.mp4")

if inputVideo.isOpened():
    ret, frame = inputVideo.read()

print(inputVideo.isOpened())

cameraMatrix = None
distCoeffs = None
markerLength = 0.05

fs = cv2.FileStorage("tutorial_camera_params.yml", cv2.FILE_STORAGE_READ)
cameraMatrix = fs.getNode("cameraMatrix").mat()
distCoeffs = fs.getNode("distCoeffs").mat()
fs.release()

detectorParams = cv2.aruco.DetectorParameters()
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

pose_data = pd.DataFrame(columns=['Time', 'Marker ID', 'tx', 'ty', 'tz', 'rx', 'ry', 'rz'])

prev_positions = {}

while True:
    ret, frame = inputVideo.read()
    if not ret:
        break
    height, width = frame.shape[:2]
    scale = 0.5
    scaled_frame = cv2.resize(frame, (int(width*scale), int(height*scale)))
    image = cv2.cvtColor(scaled_frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(image, dictionary, parameters=detectorParams)
    if ids is not None and len(ids) > 0:
        cv2.aruco.drawDetectedMarkers(scaled_frame, corners, ids)
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, markerLength, cameraMatrix, distCoeffs)
        for id, rvec, tvec in zip(ids, rvecs, tvecs):
            cv2.drawFrameAxes(scaled_frame, cameraMatrix, distCoeffs, rvec, tvec, 0.1)

            if len(tvecs) == 3:
                if tuple(id) in prev_positions:
                    prev_t, prev_p, prev_o = prev_positions[tuple(id)]
                    dt = pd.Timestamp.now() - prev_t
                    v = (tvec - prev_p) / dt.total_seconds()
                    p = tvec + v*dt.total_seconds()
                    o = rvec + (rvec - prev_o) / dt.total_seconds()
                else:
                    p, o = tvec, rvec

                intersection = np.mean(tvecs, axis=0)
                print(ids, intersection)
                rvec = np.zeros((3,1))
                tvec = np.zeros((3,1))
                imgpts, _ = cv2.projectPoints(intersection, rvec, tvec, cameraMatrix, distCoeffs)

                # draw circle at intersection point
                center = tuple(map(int, imgpts[0].ravel()))
                cv2.circle(scaled_frame, center, 5, (0, 255, 0), -1)
                #print('p:', p, 'o:', o)
                data = {'Time': pd.Timestamp.now(), 'Marker ID': int(id), 'tx': p[0][0], 'ty': p[0][1], 'tz': p[0][2], 'rx': o[0][0], 'ry': o[0][1], 'rz': o[0][2]}
                pose_data = pose_data.append(data, ignore_index=True)
                prev_positions[tuple(id)] = (pd.Timestamp.now(), p, o)

    cv2.imshow("out", scaled_frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

pose_data.to_excel("pose_data.xlsx", index=False)
inputVideo.release()
cv2.destroyAllWindows()

### ids

In [51]:
import cv2
import numpy as np
import pandas as pd

inputVideo = cv2.VideoCapture("video\whiteBorders_Trim.mp4")

if inputVideo.isOpened():
    ret, frame = inputVideo.read()

print(inputVideo.isOpened())

cameraMatrix = None
distCoeffs = None
markerLength = 0.05

fs = cv2.FileStorage("tutorial_camera_params.yml", cv2.FILE_STORAGE_READ)
cameraMatrix = fs.getNode("cameraMatrix").mat()
distCoeffs = fs.getNode("distCoeffs").mat()
fs.release()

detectorParams = cv2.aruco.DetectorParameters()
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

pose_data = pd.DataFrame(columns=['Time', 'Marker ID', 'tx', 'ty', 'tz', 'rx', 'ry', 'rz'])

prev_positions = {}
while True:
    ret, frame = inputVideo.read()
    if not ret:
        break
    height, width = frame.shape[:2]
    scale = 0.5
    scaled_frame = cv2.resize(frame, (int(width*scale), int(height*scale)))
    image = cv2.cvtColor(scaled_frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(image, dictionary, parameters=detectorParams)
    if ids is not None and len(ids) > 0:
        cv2.aruco.drawDetectedMarkers(scaled_frame, corners, ids)
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, markerLength, cameraMatrix, distCoeffs)
        detected_ids = set(ids.flatten())
        if 12 in detected_ids or 5 in detected_ids:  # check for required ids
            for id, rvec, tvec in zip(ids, rvecs, tvecs):
                cv2.drawFrameAxes(scaled_frame, cameraMatrix, distCoeffs, rvec, tvec, 0)
                print("tvecs",tvecs)
                if len(tvecs) == 3:
                    if tuple(id) in prev_positions:
                        prev_t, prev_p, prev_o = prev_positions[tuple(id)]
                        dt = pd.Timestamp.now() - prev_t
                        v = (tvec - prev_p) / dt.total_seconds()
                        p = tvec + v*dt.total_seconds()
                        o = rvec + (rvec - prev_o) / dt.total_seconds()
                    else:
                        p, o = tvec, rvec

                    # find the markers with required ids
                    if id == 12 or id == 10 or id == 13:
                        marker_1 = tvec 
                    if id == 5 or id == 8 or id == 6:
                        marker_2 = tvec
                        
                    print("marker1", marker_1,"marker2", marker_2)
                    # fix z-axis as the common axis and find the intersection point
                    marker_1_z = marker_1[:].reshape(-1)
                    marker_2_z = marker_2[:].reshape(-1)
                    # intersection = np.array([0, 0, (marker_1_z + marker_2_z) / 2])
                    intersection = 0.5*(marker_1_z + marker_2_z)
                    print("ids: ",ids, "int: ", intersection)
                    image_points = cv2.projectPoints(intersection, rvec, tvec, cameraMatrix, distCoeffs)[0][0]
                    print(image_points)
                    # Draw a circle at the intersection point (assuming we have an image called 'frame')
                    intersection_x, intersection_y = int(intersection[0]), int(intersection[1])
                    
                    cv2.circle(scaled_frame, (int(image_points[0]),int(image_points[1])), 5, (255, 0, 255), -1)
            
        # Draw the detected aruco markers and axes
        ee = cv2.aruco.drawDetectedMarkers(scaled_frame, corners, ids)
        for i in range(len(ids)):
            ee = cv2.drawFrameAxes(ee, cameraMatrix, distCoeffs, rvecs[i], tvecs[i], 0.1)

    # Display the resulting frame
    cv2.imshow('frame', ee)


    # Exit loop if 'esc' key is pressed
    key = cv2.waitKey(1)
    if key == 27:
        break
# Release the video capture object and close all windows
inputVideo.release()
cv2.destroyAllWindows()

True
tvecs [[[-0.43416566 -0.03145778  1.57225056]]

 [[-0.71812526  0.02275633  1.543125  ]]

 [[-0.51318101 -0.08877032  1.32194208]]]
marker1 [[-0.43416566 -0.03145778  1.57225056]] marker2 [[-0.70251845  0.02528414  1.50230057]]
ids:  [[13]
 [ 5]
 [11]] int:  [-0.56834206 -0.00308682  1.53727557]
[[ 23741.16295461 -20467.29882362]]


TypeError: only size-1 arrays can be converted to Python scalars